## Introduction to choice-learn's modelling

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""

import sys

sys.path.append("../../")

import numpy as np
import pandas as pd

## Summary

- [Example 1: ConditionalMNL with Swissmetro](#example-1-swissmetro)
    - [A few words on c-MNL formulation](#the-conditional-logit-model)
    - [Instantiation and estimation with Choice-Learn](#conditional-logit-estimation-with-choice-learn)
- [Example 2: ConditionalMNL with ModeCanda](#example-2-the-modecanada-dataset)
    - [Utility Formulation](#utility-formulation)
    - [Model Specification](#model-formulation)
    - [Lighter method for specification](#faster-specification)
    - [Comparison with other implementations](#comparison-with-other-implementations-results)
    - [Get utility estimation and probabilities](#estimate-utility--probabilities)
    - [Using Gradient Descent Optimizers](#using-gradient-descent-optimizers)
    

For model customization and more explanation on ChoiceModel and the endpoints, you can go [here](./custom_model.ipynb)

## Example 1: SwissMetro

The choice-learn package offers a high level API to conceive and estimate discrete choice models. Several models are ready to be used, you can check the list [here](../README.md). If you want to create your own model or another one that is not in the list, the lower level API can help you. Check the notebook [here](./4_model_customization.ipynb).

Let's begin this tutorial with the estimation of a Conditional Logit Model on the SwissMetro dataset[3].
It follows the specifications described in [PyLogit](https://github.com/timothyb0912/pylogit/blob/master/examples/notebooks/Main%20PyLogit%20Example.ipynb) and [Biogeme](https://github.com/mncosta/biogeme_tutorial/blob/master/6.1-MultinomialLogitAndProbitModels/01-biogeme-basics.ipynb).


First, we download our data as a ChoiceDataset. See the [data management tutorial](./choice_learn_introduction_data.ipynb) first if needed.

In [ ]:
from choice_learn.datasets import load_swissmetro
swiss_dataset = load_swissmetro(preprocessing="tutorial")
print(swiss_dataset.summary())

### The Conditional Logit model

The conditional Logit [2] specifies a linear utility for each features of item $i$:
$$
U(i) = \sum_{features} a(i) * feature(i)
$$
The probability to choose $i$ among the set of available alternatives $\mathcal{A}$ is then:

$$
\mathbb{P}(i) = \frac{e^{U(i)}}{\sum_{j \in \mathcal{A}} e^{U(j)}}
$$

With the SwissMetro dataset we are trying to predict a customer mean of transport among train, swissmetro and car from the features:
- TT (transit time)
- CO (cost)
- HE (headway)
- Survey (where the survey took place)
- Luggage number
- seats configuration in the swissmetro
- first class or not

An important step is to define the right utility function for the model to fit well the dataset. Let's take the following formulation defined in PyLogit:

- $
U(train) = \beta_{train}^{inter} + \beta^{tt}_{train/sm} \cdot TT(train) + \beta^{co}_{train} \cdot CO(train) + \beta^{he}_{train} \cdot HE(train) + \beta^{survey} \cdot SV(train)
$

- $
U(sm) = \beta_{sm}^{inter} + \beta^{tt}_{train/sm} \cdot TT(sm) + \beta^{co}_{sm} \cdot CO(sm) + \beta^{he}_{sm} \cdot HE(sm) + \beta^{survey} \cdot SV(sm) + \beta^{seat} \cdot SEAT(sm) + \beta^{first\_class} \cdot FC(sm)
$

- $
U(car) = \beta^{tt}_{car} \cdot TT(car) + \beta^{co}_{car} \cdot CO(car) + \beta^{luggage==1} \cdot \mathbb{1}_{Luggage==1} + \beta^{luggage>1} \cdot \mathbb{1}_{Luggage>1}
$

Note that we want to estimate:

- one $\beta^{tt}_{train/sm}$ **shared** train and sm items and one $\beta^{tt}_{car}$ for the car item
- one $\beta^{co}$ coefficient for **each** item.
- one $\beta^{inter}$ and $\beta^{he}$ for train and sm, and zeroed for the car alternative
- one $\beta^{survey}$, $\beta^{seat}$, $\beta^{first\_class}$, $\beta^{luggage==1}$ and $\beta^{luggage>1}$ shared or not by different items

To build a model, we need to specify for each weight $\beta$:
- the name of the feature it goes with:
    - it must match the feature name in the ChoiceDataset
    - "intercept" is the standardized name used for intercept, pay attention not to override it
- items_indexes: the items concerned, as indexed in the ChoiceDataset
- (optionally) a unique weight name

> **Attention**
> 
> *add_coefficients* is to be used to get one coefficient by given items_indexes\
> *add_shared_coefficients* is to be used to get on coefficient that is used for utility of all given items_indexes

Here is how to create a model following our defined utility functions:

### Conditional Logit Estimation with Choice-Learn

In [ ]:
from choice_learn.models import ConditionalMNL

In [ ]:
# Initialization of the model
swiss_model = ConditionalMNL(optimizer="lbfgs")

# Intercept for train & sm
swiss_model.add_coefficients(feature_name="intercept", items_indexes=[0, 1])
# beta_he for train & sm
swiss_model.add_coefficients(feature_name="headway",
                             items_indexes=[0, 1],
                             coefficient_name="beta_he")
# beta_co for all items
swiss_model.add_coefficients(feature_name="cost",
                             items_indexes=[0, 1, 2])
# beta first_class for train
swiss_model.add_coefficients(feature_name="regular_class",
                             items_indexes=[0])
# beta seats for train
swiss_model.add_coefficients(feature_name="seats", items_indexes=[1])
# betas luggage for car
swiss_model.add_coefficients(feature_name="single_luggage_piece",
                             items_indexes=[2],
                             coefficient_name="beta_luggage=1")
swiss_model.add_coefficients(feature_name="multiple_luggage_piece",
                             items_indexes=[2],
                             coefficient_name="beta_luggage>1")
# beta TT only for car
swiss_model.add_coefficients(feature_name="travel_time",
                             items_indexes=[2],
                             coefficient_name="beta_tt_car")

# betas TT and HE shared by train and sm
swiss_model.add_shared_coefficient(feature_name="travel_time",
                                   items_indexes=[0, 1])
swiss_model.add_shared_coefficient(feature_name="train_survey",
                                   items_indexes=[0, 1],
                                   coefficient_name="beta_survey")


In [ ]:
# Estimation of the model
history = swiss_model.fit(swiss_dataset, get_report=True)

Onece the model is estimated, we can look at the weights with the .trainable_weights argument:

In [ ]:
swiss_model.trainable_weights

[<tf.Variable 'beta_intercept:0' shape=(1, 2) dtype=float32, numpy=array([[-1.2929311, -0.5025746]], dtype=float32)>,
 <tf.Variable 'beta_he:0' shape=(1, 2) dtype=float32, numpy=array([[-0.31433567, -0.37731853]], dtype=float32)>,
 <tf.Variable 'beta_cost:0' shape=(1, 3) dtype=float32, numpy=array([[-0.56176245, -0.28167567, -0.5138463 ]], dtype=float32)>,
 <tf.Variable 'beta_regular_class:0' shape=(1, 1) dtype=float32, numpy=array([[0.5650174]], dtype=float32)>,
 <tf.Variable 'beta_seats:0' shape=(1, 1) dtype=float32, numpy=array([[-0.7824476]], dtype=float32)>,
 <tf.Variable 'beta_luggage=1:0' shape=(1, 1) dtype=float32, numpy=array([[0.4227598]], dtype=float32)>,
 <tf.Variable 'beta_luggage>1:0' shape=(1, 1) dtype=float32, numpy=array([[1.4139806]], dtype=float32)>,
 <tf.Variable 'beta_tt_car:0' shape=(1, 1) dtype=float32, numpy=array([[-0.7229834]], dtype=float32)>,
 <tf.Variable 'beta_travel_time:0' shape=(1, 1) dtype=float32, numpy=array([[-0.69901353]], dtype=float32)>,
 <tf.Var

We can easily acces the negative log likelihood value for the training dataset or another one using the .evaluate() method:

In [ ]:
len(swiss_dataset) * swiss_model.evaluate(swiss_dataset)

<tf.Tensor: shape=(), dtype=float32, numpy=5156.3345>

If yoou set get_report to True in .fit, the model automatically creates a report for each of the coefficient, with its estimation, its standard deviation and more:

In [ ]:
swiss_model.report

We find the same results (estimation of parameters and negative log-likelihood) as the PyLogit package.

## Example #2: the ModeCanada Dataset


### Utility formulation

Let's reproduce a common example from [Torch-Choice](https://gsbdbi.github.io/torch-choice/conditional_logit_model_mode_canada/) on the ModeCanada [1] dataset:
$$
U(i, c) = \beta^{inter}_i + \beta^{price} \cdot price(i, c) + \beta^{freq} \cdot freq(i, c) + \beta^{ovt} \cdot ovt(i, c) + \beta^{income}_i \cdot income(c) + \beta^{ivt}_i \cdot ivt(i, c) + \epsilon(i, c)
$$

A description of the dataset and the features can be found in [1]. We want to predict the future mean of transport from [train, air, bus, car] using frequence, price, in-vehicule transport time (ivt) and out-of-vehicule transport time (ovt).

### Model Formulation

In [ ]:
# If you want to check what's in the dataset:
from choice_learn.datasets import load_modecanada

transport_df = load_modecanada(as_frame=True)
transport_df.head()

We want to estimate:

- one $\beta^{price}$, $\beta^{freq}$ and $\beta^{ovt}$ coefficient. They are **shared** by all items.
- one $\beta^{ivt}$ coefficient for **each** item.
- one $\beta^{inter}$ and $\beta^{income}$ coefficient for **each** item, with **additional constraint** to be 0 for the first item (air).

One notes that it makes sense to include an intercept $\beta^{inter}$ for each item since $ivt(i, c)$ and $income(c)$ depends on each choice $c$.

Additionally to previous example we manually specify the weights names:

In [ ]:
# Loading the ChoiceDataset
canada_dataset = load_modecanada(as_frame=False, preprocessing="tutorial")

print(canada_dataset.summary())

In [ ]:
from choice_learn.models import ConditionalMNL

# Initialization of the model
model = ConditionalMNL()

# Creation of the different weights:

# shared_coefficient add one coefficient that is used for all items specified in the items_indexes:
# Here, cost, freq and ovt coefficients are shared between all items
model.add_shared_coefficient(feature_name="cost", items_indexes=[0, 1, 2, 3])
# You can specify you own coefficient name
model.add_shared_coefficient(feature_name="freq",
                             coefficient_name="beta_frequence",
                             items_indexes=[0, 1, 2, 3])
model.add_shared_coefficient(feature_name="ovt", items_indexes=[0, 1, 2, 3])

# ivt is added for each item:
model.add_coefficients(feature_name="ivt", items_indexes=[0, 1, 2, 3])

# add_coefficients adds one coefficient for each specified item_index
# intercept, and income are added for each item except the first one that needs to be zeroed
model.add_coefficients(feature_name="intercept", items_indexes=[1, 2, 3])
model.add_coefficients(feature_name="income", items_indexes=[1, 2, 3])

In [ ]:
history = model.fit(canada_dataset, get_report=True, verbose=2)

In [ ]:
model.trainable_weights

In [ ]:
print("The average neg-loglikelihood is:", model.evaluate(canada_dataset).numpy())
print("The total neg-loglikelihood is:", model.evaluate(canada_dataset).numpy()*len(canada_dataset))

In [ ]:
model.report

### Faster Specification

A faster specification can be done using a dictionnary. It follows torch-choice method to create conditional logit models.
The parameters dict needs to be as follows:
- The key is the feature name
- The value is the mode. Currently three modes are available:
    - constant: the learned coefficient is shared by all items
    - item: one coefficient by item is estimated, the value for the item at index 0 is set to 0
    - item-full: one coefficient by item is estimated

In order to create the same model for the ModeCanada dataset, it looks as follows:

In [ ]:
# Instantiation with the coefficients dictionnary
coefficients = {"income": "item",
 "cost": "constant",
 "freq": "constant",
 "ovt": "constant",
 "ivt": "item-full",
 "intercept": "item"}

# Instantiation of the model
cmnl = ConditionalMNL(coefficients=coefficients, epochs=1000, optimizer="lbfgs")

In [ ]:
history = cmnl.fit(canada_dataset)
print(cmnl.trainable_weights)
print(cmnl.evaluate(canada_dataset).numpy())

### Comparison with other implementations results

In [ ]:
import tensorflow as tf

# Here are the values obtained in the references:
gt_weights = [
    tf.constant([[-0.0890796, -0.0279925, -0.038146]]),
    tf.constant([[-0.0333421]]),
    tf.constant([[0.0925304]]),
    tf.constant([[-0.0430032]]),
    tf.constant([[0.0595089, -0.00678188, -0.00645982, -0.00145029]]),
    tf.constant([[0.697311, 1.8437, 3.27381]]),
]
gt_model = ConditionalMNL(coefficients=coefficients)
gt_model.instantiate(canada_dataset)
canada_dataset
# Here we estimate the negative log-likelihood with these coefficients (also, we obtain same value as in those papers):
gt_model.trainable_weights = gt_weights
print("'Ground Truth' Negative LogLikelihood:", gt_model.evaluate(canada_dataset) * len(canada_dataset))

### Estimate Utility & probabilities
In order to estimate the utilities, use the .predict_utility() method. In order to estimate the probabilities, use the .predict_probas() method.


In [ ]:
print("Utilities of each item for the first 5 sessions:", cmnl.compute_batch_utility(*canada_dataset.batch[:5]))

In [ ]:
print("Purchase probability of each item for the first 5 sessions:", cmnl.predict_probas(canada_dataset)[:5])

### Using Gradient Descent Optimizers

For very large datasets that do not fit entirely in the memory, we have to work with data batches. In this case, the LBFGS method is usually not the best choice. In those cases, we will prefer stochastic gradient descent optimizers.

In this case, it is possible to obtain the same coefficients estimation, also it is a little tricky to get it quickly. We need to adjust the learning rate over time for the optimization not to be too slow.
L-BFGS is more efficient for small dataset - Gradient Descent for large ones !

In [ ]:
cmnl = ConditionalMNL(coefficients=coefficients, optimizer="Adam", epochs=2000, batch_size=-1)
history = cmnl.fit(canada_dataset)
cmnl.optimizer.lr = cmnl.optimizer.lr / 5
cmnl.epochs = 4000
history2 = cmnl.fit(canada_dataset)
cmnl.optimizer.lr = cmnl.optimizer.lr  / 10
cmnl.epochs = 20000
history3 = cmnl.fit(canada_dataset)

It can be useful to look at the loss (negative loglikelyhood) over time to see how the estimation goes:

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history["train_loss"])
plt.title("First part of the gradient descent.")
plt.subplot(1, 2, 2)
plt.plot(history2["train_loss"] + history3["train_loss"])
plt.title("Second and third part of the gradient descent.")

In [ ]:
cmnl.trainable_weights

In [ ]:
cmnl.evaluate(canada_dataset)

### References

[1] ModeCanada dataset in *Application and interpretation of nested logit models of intercity mode choice*, Christophier, V. F.; Koppelman, S. (1993)\
[2] Conditional MultinomialLogit, Train, K.; McFadden, D.; Ben-Akiva, M. (1987)\
[3] Siwssmetro dataset in *The acceptance of modal innovation: The case of Swissmetro*, Bierlaire, M.; Axhausen, K.; Abay, G (2001)\